# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
!python -m pip install semantic-kernel==0.4.3.dev0

In [7]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion(ai_model_id="gpt35", api_key=api_key, org_id=org_id))


Let's define a prompt outlining a dialogue chat bot.

In [8]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [9]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5)

Initialize your context

In [10]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [11]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Sure, what genre are you interested in?


Update the history with the output

In [12]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure, what genre are you interested in?



Keep Chatting!

In [13]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [14]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: How about "The Greeks: A Portrait of Self and Others" by Paul Cartledge or "The Birth of Classical Europe: A History from Troy to Augustine" by Simon Price and Peter Thonemann?


In [15]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Greeks: A Portrait of Self and Others" is a comprehensive overview of ancient Greek culture, society, and history. It explores the Greeks' relationship with themselves and with other cultures, and how their ideas and values have influenced Western civilization. "The Birth of Classical Europe: A History from Troy to Augustine" is a more detailed account of the development of ancient Greece and Rome, and how they shaped the world we live in today. It covers everything from the Trojan War to the rise of Christianity.


In [16]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: You will learn about the origins of ancient Greece, its political and social structures, its cultural achievements, and its interactions with other civilizations. You will also gain insight into the lives of famous Greeks such as Socrates, Plato, and Aristotle, and their contributions to philosophy and science. Additionally, you will learn about the wars and conflicts that shaped Greek history, including the Persian Wars and the Peloponnesian War.


In [17]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Sure, here are some more book suggestions:
- "The Peloponnesian War" by Thucydides
- "The Iliad" and "The Odyssey" by Homer
- "The Histories" by Herodotus
- "The Republic" by Plato
- "The Nicomachean Ethics" by Aristotle
- "The Greek Way" by Edith Hamilton
- "The Greeks and Greek Civilization" by Jacob Burckhardt
- "The Greek Myths" by Robert Graves
- "The Oxford History of Greece and the Hellenistic World" edited by John Boardman, Jasper Griffin, and Oswyn Murray.


After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [18]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure, what genre are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: How about "The Greeks: A Portrait of Self and Others" by Paul Cartledge or "The Birth of Classical Europe: A History from Troy to Augustine" by Simon Price and Peter Thonemann?

User: that sounds interesting, what is it about?
ChatBot: "The Greeks: A Portrait of Self and Others" is a comprehensive overview of ancient Greek culture, society, and history. It explores the Greeks' relationship with themselves and with other cultures, and how their ideas and values have influenced Western civilization. "The Birth of Classical Europe: A History from Troy to Augustine" is a more detailed account of the development of ancient Greece and Rome, and how they shaped the world we live in today. It covers everything from the Trojan War to the rise of Christianity.

User: if I read that book, what exac